# Consulta à API da SeFaz-SP

1) Consulta as informações de execução financeira e orçamentária da Secretaria da Fazenda e Planejamento do Estado de São Paulo, através da API pública, com uso de requests.

2) As informações de interesse, em formato XML, são extraídas com a bibliotaca Beautiful Soup.

3) Na sequência, usa-se o Pandas para a criação do DataFrame e gravação dos dados em arquivo csv.


In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

# 27000 - MPSP
# 03000 - TJSP
# 42000 - DPSP

dados_despesas = []
orgao = '42000'
anos = [2013, 2023]

In [2]:
def consulta_api(orgao, ano):
    url = "https://webservices.fazenda.sp.gov.br/WSTransparencia/TransparenciaServico.asmx?op=ConsultarDespesas"

    headers = {'Content-Type': 'text/xml'}
    payload = f"""<?xml version=\"1.0\" encoding=\"utf-8\"?>\n
              <soap:Envelope xmlns:xsi=\
              "http://www.w3.org/2001/XMLSchema-instance\" \
              xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\" \
              xmlns:soap=\"http://schemas.xmlsoap.org/soap/envelope/\">\n
              <soap:Body>\n
              <ConsultarDespesas xmlns=\"http://fazenda.sp.gov.br/wstransparencia\">\n
              <ano>{ano}</ano>\n
              <codigoOrgao>{orgao}</codigoOrgao>\n
              <codigoFonteRecursos>Detalhado</codigoFonteRecursos>\n
              <flagCredor>1</flagCredor>\n
              <flagEmpenhado>1</flagEmpenhado>\n
              <flagLiquidado>1</flagLiquidado>\n
              <flagPago>1</flagPago>\n
              </ConsultarDespesas>\n
              </soap:Body>\n
              </soap:Envelope>"""

    resposta = requests.request("POST", url, headers=headers, data=payload, verify=False)
    print(f"Ano: {ano}: Status: {resposta.status_code}")
    return resposta.text # unicode

In [3]:
def gera_lista(ano, despesas, dados_despesas):
    bs_despesas = BeautifulSoup(despesas)
    nome_orgao = bs_despesas.find_all('codigonomeorgao')
    fonte = bs_despesas.find_all('codigonomefonterecursos')
    tipo_licitacao = bs_despesas.find_all('codigonometipolicitacao')
    favorecido = bs_despesas.find_all('cgccpffavorecido')
    natureza = bs_despesas.find_all('naturezadespesanomeitem')
    empenhado = bs_despesas.find_all('valorempenhado')
    liquidado = bs_despesas.find_all('valorliquidado')
    pago = bs_despesas.find_all('valorpago')
    anos_anteriores = bs_despesas.find_all('valorpagoanosanteriores')
    for n in range(0,len(nome_orgao)):
        linhas = [ano, nome_orgao[n].get_text(), fonte[n].get_text(), tipo_licitacao[n].get_text(), 
                  favorecido[n].get_text(), natureza[n].get_text(), empenhado[n].get_text(), 
                  liquidado[n].get_text(), pago[n].get_text(), anos_anteriores[n].get_text()]
        dados_despesas.append(linhas)
    return dados_despesas

In [4]:
def converte_moeda (str_numero):
    """Converte valores monetários para float com duas casas decimais"""
    try:
        str_numero = str_numero.replace('.', '').replace(',', '.')
        resultado = round(float(str_numero), 2)
        return resultado
    except ValueError:
        return str_numero

In [5]:
import warnings
warnings.filterwarnings("ignore")
for ano in range (anos[0], anos[1]+1):
    despesas = consulta_api(orgao, str(ano))
    gera_lista(ano, despesas, dados_despesas)
warnings.resetwarnings()

Ano: 2013: Status: 200
Ano: 2014: Status: 200
Ano: 2015: Status: 200
Ano: 2016: Status: 200
Ano: 2017: Status: 200
Ano: 2018: Status: 200
Ano: 2019: Status: 200
Ano: 2020: Status: 200
Ano: 2021: Status: 200
Ano: 2022: Status: 200
Ano: 2023: Status: 200


In [6]:
# Criação do Dataframe
colunas = ['Ano', 'Órgão', 'Fonte de recursos', 'Tipo de licitação', 'Favorecido','Natureza',
           'Empenhado', 'Liquidado', 'Pago', 'Anos anteriores']

df_despesas = pd.DataFrame(dados_despesas, columns=colunas)

# Eliminação das linhas de totalização
df_despesas['Órgão'].replace('', np.nan, inplace=True)
df_despesas.dropna(subset = ['Órgão'], inplace=True)
df_despesas.to_excel(f'despesas_{orgao}.xlsx', index=None)
df_despesas.sample(5)

,Ano,Órgão,Fonte de recursos,Tipo de licitação,Favorecido,Natureza,Empenhado,Liquidado,Pago,Anos anteriores
2154,2015,42000 - DEFENSORIA PUBLICA DO ESTADO,002 - RECURSOS VINCULADOS ESTADUAIS,7 - PREGÃO,14543349000184 - ARPEL ARTE EM PAPEL LTDA - EPP,33903983 - SERVICOS GRAFICOS,"4.500,00","4.500,00","4.500,00","0,00"
296,2013,42000 - DEFENSORIA PUBLICA DO ESTADO,002 - RECURSOS VINCULADOS ESTADUAIS,5 - DISPENSA DE LICITACAO,21472644824 - MONIKA WERONIKA DOWBOR DA SILVA,33903611 - OUTRAS REMUNERACOES DE SERVICOS PES...,"1.530,00","1.530,00","1.530,00","0,00"
13255,2023,42000 - DEFENSORIA PUBLICA DO ESTADO,"176020 - RECURSOS DE EMOLUMENTOS, TAXAS E CUST...",9 - INDEPEND LICITACAO,08424315600 - PATRICIA MARIA LIZ DE OLIVEIRA,33901401 - DIARIAS PESSOAL CIVIL,"44.411,32","44.411,32","41.766,61","0,00"
1014,2014,42000 - DEFENSORIA PUBLICA DO ESTADO,002 - RECURSOS VINCULADOS ESTADUAIS,5 - DISPENSA DE LICITACAO,00741304473 - JOYCE GUERRA,33903611 - OUTRAS REMUNERACOES DE SERVICOS PES...,"370,00","370,00","370,00","0,00"
11344,2022,42000 - DEFENSORIA PUBLICA DO ESTADO,003 - RECURSOS VINCULADOS-FUNDO ESPECIAL DE DE...,9 - INDEPEND LICITACAO,05230755610 - CARLOS ALEXANDRE LEMOS OLIVEIRA,33909301 - INDENIZACOES E RESTITUICOES DIVERSAS,"1.499,00","1.499,00","1.499,00","0,00"


In [7]:
df_despesas['Empenhado'] = df_despesas['Empenhado'].apply(lambda x:converte_moeda(x))
df_despesas['Liquidado'] = df_despesas['Liquidado'].apply(lambda x:converte_moeda(x))
df_despesas['Pago'] = df_despesas['Pago'].apply(lambda x:converte_moeda(x))
df_despesas['Anos anteriores'] = df_despesas['Anos anteriores'].apply(lambda x:converte_moeda(x))
df_despesas.sample(5)

,Ano,Órgão,Fonte de recursos,Tipo de licitação,Favorecido,Natureza,Empenhado,Liquidado,Pago,Anos anteriores
13137,2023,42000 - DEFENSORIA PUBLICA DO ESTADO,"176020 - RECURSOS DE EMOLUMENTOS, TAXAS E CUST...",7 - PREGÃO,03619767000515 - TORINO INFORMATICA LTDA,44905220 - EQUIPAMENTOS DE TECNOLOGIA DA INFOR...,2250083.28,2250083.28,2250083.28,0.00
13193,2023,42000 - DEFENSORIA PUBLICA DO ESTADO,"176020 - RECURSOS DE EMOLUMENTOS, TAXAS E CUST...",9 - INDEPEND LICITACAO,24785608870 - HARLEY DE OLIVEIRA MORAES,33900841 - AUXILIO FUNERAL - OUTROS PODERES,4630.36,4630.36,4630.36,0.00
898,2014,42000 - DEFENSORIA PUBLICA DO ESTADO,002 - RECURSOS VINCULADOS ESTADUAIS,0 - ADIANTAMENTO,22107739897 - MARIANA PAGANO GIL,33903992 - DESPESAS MIUDAS E DE PRONTO PAGAMENTO,7265.32,7265.32,7265.32,0.00
4607,2018,42000 - DEFENSORIA PUBLICA DO ESTADO,002 - RECURSOS VINCULADOS ESTADUAIS,9 - INDEPEND LICITACAO,26166847000134 - ALMEIDA SOCIEDADE INDIVIDUAL ...,33903988 - HONORARIOS ADVOCAT. ASSIST. JUDICIA...,9128.76,9128.76,6997.08,1227.34
2640,2016,42000 - DEFENSORIA PUBLICA DO ESTADO,002 - RECURSOS VINCULADOS ESTADUAIS,0 - ADIANTAMENTO,29222434862 - MICHELLE BOAVENTURA CORDEIRO,33903992 - DESPESAS MIUDAS E DE PRONTO PAGAMENTO,4410.74,4410.74,4410.74,0.00


In [8]:
despesas_por_natureza = df_despesas.groupby(['Ano','Natureza'])['Pago'].sum()
despesas_por_natureza

Ano   Natureza                                               
2013  31900701 - CONTRIBUICAO PATRONAL                           2.525384e+04
      31901112 - PESSOAL CIVIL PAGO PELA UNIDADE                 1.689391e+08
      31901129 - PESSOAL CIVIL PAGO PELA UNIDADE-13 SALARIO      1.422255e+07
      31901131 - PESSOAL CIVIL PAGO PELA UNIDADE-1/3 FERIAS      4.038591e+06
      31901136 - ABONO DE PERMANENCIA                            8.426421e+04
                                                                     ...     
2023  33909301 - INDENIZACOES E RESTITUICOES DIVERSAS            3.143082e+06
      33913996 - TAXA DE ADMINISTRACAO A SAO PAULO PREV-SPPRE    3.645958e+05
      44905220 - EQUIPAMENTOS DE TECNOLOGIA DA INFORMACãO        3.307414e+06
      44905232 - MOBILIARIO EM GERAL                             1.605996e+05
      44905234 - OUTROS EQUIPAMENTOS E MATERIAL PERMANENTE       9.896102e+04
Name: Pago, Length: 1036, dtype: float64

In [9]:
# Soma das despesas por ano e natureza
grupo_despesas = df_despesas.groupby(['Ano', 'Natureza'])['Pago'].sum().reset_index()

# Soma total de despesas por ano
soma_total_por_ano = grupo_despesas.groupby('Ano')['Pago'].transform('sum')

# Porcentagem das despesas para cada ano
grupo_despesas['Porcentagem'] = round((grupo_despesas['Pago'] / soma_total_por_ano * 100), 2)

display(grupo_despesas)

,Ano,Natureza,Pago,Porcentagem
0,2013,31900701 - CONTRIBUICAO PATRONAL,2.525384e+04,0.00
1,2013,31901112 - PESSOAL CIVIL PAGO PELA UNIDADE,1.689391e+08,31.21
2,2013,31901129 - PESSOAL CIVIL PAGO PELA UNIDADE-13 ...,1.422255e+07,2.63
3,2013,31901131 - PESSOAL CIVIL PAGO PELA UNIDADE-1/3...,4.038591e+06,0.75
4,2013,31901136 - ABONO DE PERMANENCIA,8.426421e+04,0.02
...,...,...,...,...
1031,2023,33909301 - INDENIZACOES E RESTITUICOES DIVERSAS,3.143082e+06,0.39
1032,2023,33913996 - TAXA DE ADMINISTRACAO A SAO PAULO P...,3.645958e+05,0.05
1033,2023,44905220 - EQUIPAMENTOS DE TECNOLOGIA DA INFOR...,3.307414e+06,0.41
1034,2023,44905232 - MOBILIARIO EM GERAL,1.605996e+05,0.02


In [10]:
# Arquivo para o Flourish
grupo_despesas.to_csv(f'despesas_por_natureza_porcentagem_{orgao}.csv', index=False, decimal=',')

Consulta web:https://www.fazenda.sp.gov.br/SigeoLei131/Paginas/FlexConsDespesa.aspx

Link de acesso ao webservice da Fazenda: https://webservices.fazenda.sp.gov.br/WSTransparencia/TransparenciaServico.asmx

Manual técnico: https://portal.fazenda.sp.gov.br/acessoinformacao/Downloads/Webservice/Manual%20do%20Webservice.pdf

Conversão do XML em Dataframe baseado em: https://www.geeksforgeeks.org/convert-xml-structure-to-dataframe-using-beautifulsoup-python/

Criado em 11 jul. 2021 e atualizado em 12 nov. 2023.